In [1]:
import pandas as pd
import numpy as np
import findspark
findspark.init('C:\spark\spark-3.1.2-bin-hadoop3.2')
import pyspark

from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark import SparkConf
from pyspark.sql.functions import col
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)


In [2]:
df = spark.read.csv("zoo.csv", header=True, inferSchema=True)

In [3]:
df.show()

+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+
|animal_name|hair|feathers|eggs|milk|airborne|aquatic|predator|toothed|backbone|breathes|venomous|fins|legs|tail|domestic|catsize|class_type|
+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+
|   aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|         1|
|   antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|   4|   1|       0|      1|         1|
|       bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|   0|   1|       0|      0|         4|
|       bear|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|         1|
|     

In [4]:
from pyspark.sql.functions import col,when

In [6]:
df2 = df.withColumn("IsMammal", when(col("class_type") ==1,"True").otherwise("False"))

In [7]:
df2.show()

+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+--------+
|animal_name|hair|feathers|eggs|milk|airborne|aquatic|predator|toothed|backbone|breathes|venomous|fins|legs|tail|domestic|catsize|class_type|IsMammal|
+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+--------+
|   aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|         1|    True|
|   antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|   4|   1|       0|      1|         1|    True|
|       bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|   0|   1|       0|      0|         4|   False|
|       bear|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|     

In [8]:
df2.select("class_type","IsMammal").show()

+----------+--------+
|class_type|IsMammal|
+----------+--------+
|         1|    True|
|         1|    True|
|         4|   False|
|         1|    True|
|         1|    True|
|         1|    True|
|         1|    True|
|         4|   False|
|         4|   False|
|         1|    True|
|         1|    True|
|         2|   False|
|         4|   False|
|         7|   False|
|         7|   False|
|         7|   False|
|         2|   False|
|         1|    True|
|         4|   False|
|         1|    True|
+----------+--------+
only showing top 20 rows



In [9]:
from pyspark.ml.feature import RFormula

In [15]:
supervised = RFormula(formula = "IsMammal ~ . + animal_name:hair + animal_name:feathers + animal_name:eggs + animal_name:milk + animal_name:airborne + animal_name:aquatic + animal_name:predator + animal_name:toothed + animal_name:backbone + animal_name:breathes + animal_name:venomous + animal_name:fins + animal_name:legs + animal_name:tail + animal_name:domestic + animal_name:catsize + animal_name:class_type")

In [16]:
fittedRF = supervised.fit(df2)
preparedDF = fittedRF.transform(df2)
preparedDF.show()

+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+--------+--------------------+-----+
|animal_name|hair|feathers|eggs|milk|airborne|aquatic|predator|toothed|backbone|breathes|venomous|fins|legs|tail|domestic|catsize|class_type|IsMammal|            features|label|
+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+--------+--------------------+-----+
|   aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|         1|    True|(1816,[1,99,102,1...|  1.0|
|   antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|   4|   1|       0|      1|         1|    True|(1816,[2,99,102,1...|  1.0|
|       bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|   0|  

In [17]:
train,test = preparedDF.randomSplit([0.7,0.3])

In [18]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol = "label",featuresCol = "features")

In [19]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The

In [20]:
fittedLR = lr.fit(train)

In [21]:
fittedLR.transform(train).select("AnimalName","IsMammal","label","prediction").show(100)

AnalysisException: cannot resolve '`AnimalName`' given input columns: [IsMammal, airborne, animal_name, aquatic, backbone, breathes, catsize, class_type, domestic, eggs, feathers, features, fins, hair, label, legs, milk, predator, prediction, probability, rawPrediction, tail, toothed, venomous];
'Project ['AnimalName, IsMammal#144, label#1787, prediction#2218]
+- Project [animal_name#16, hair#17, feathers#18, eggs#19, milk#20, airborne#21, aquatic#22, predator#23, toothed#24, backbone#25, breathes#26, venomous#27, fins#28, legs#29, tail#30, domestic#31, catsize#32, class_type#33, IsMammal#144, features#1724, label#1787, rawPrediction#2162, probability#2188, UDF(rawPrediction#2162) AS prediction#2218]
   +- Project [animal_name#16, hair#17, feathers#18, eggs#19, milk#20, airborne#21, aquatic#22, predator#23, toothed#24, backbone#25, breathes#26, venomous#27, fins#28, legs#29, tail#30, domestic#31, catsize#32, class_type#33, IsMammal#144, features#1724, label#1787, rawPrediction#2162, UD